## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-28-20-50-43 +0000,bbc,Minneapolis mourns two children killed in shoo...,https://www.bbc.com/news/articles/cy40znde9eko...
1,2025-08-28-20-48-47 +0000,startribune,Live: U.S. attorney says no clear motive for s...,https://www.startribune.com/robin-westman-annu...
2,2025-08-28-20-46-22 +0000,nypost,Stocks hit all-time highs as US economy grows ...,https://nypost.com/2025/08/28/business/stocks-...
3,2025-08-28-20-42-19 +0000,nyt,RFK Jr. Sought to Fire CDC Director Susan Mona...,https://www.nytimes.com/2025/08/28/health/rfk-...
4,2025-08-28-20-42-02 +0000,nypost,House GOP Whip Tom Emmer calls for repeal of M...,https://nypost.com/2025/08/28/us-news/house-go...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2391/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
54,trump,48
0,minneapolis,16
60,school,15
5,shooting,13
588,new,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
10,2025-08-28-20-35-00 +0000,wsj,"Fed governor Lisa Cook sued President Trump, s...",https://www.wsj.com/economy/central-banking/li...,131
246,2025-08-28-01-00-00 +0000,wsj,"In his first term, administration officials re...",https://www.wsj.com/politics/policy/in-trumps-...,125
131,2025-08-28-14-18-31 +0000,cbc,U.S. Fed governor Lisa Cook suing Trump admini...,https://www.cbc.ca/news/world/trump-lisa-cook-...,106
110,2025-08-28-15-24-39 +0000,bbc,US Fed Governor Lisa Cook sues Trump over his ...,https://www.bbc.com/news/articles/c1dxl6ry4y3o...,103
141,2025-08-28-13-46-00 +0000,wsj,"President Trump fired Robert Primus, a board m...",https://www.wsj.com/politics/policy/trump-fire...,98


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
10,131,2025-08-28-20-35-00 +0000,wsj,"Fed governor Lisa Cook sued President Trump, s...",https://www.wsj.com/economy/central-banking/li...
43,73,2025-08-28-19-06-50 +0000,nypost,Minneapolis school shooting victim’s mother wa...,https://nypost.com/2025/08/28/us-news/minneapo...
225,71,2025-08-28-03-19-08 +0000,nypost,Russian ballistic missiles hit Kyiv in large-s...,https://nypost.com/2025/08/27/world-news/russi...
220,48,2025-08-28-04-20-00 +0000,wsj,"Susan Monarez, the director of the Centers for...",https://www.wsj.com/politics/policy/cdc-direct...
300,44,2025-08-27-21-29-33 +0000,nypost,Zohran Mamdani would easily win NYC mayor race...,https://nypost.com/2025/08/27/us-news/zohran-m...
313,37,2025-08-27-21-01-42 +0000,nypost,Angry Cracker Barrel workers dish on skimpy pa...,https://nypost.com/2025/08/27/business/angry-c...
12,35,2025-08-28-20-32-41 +0000,nypost,"UK, EU accuse Putin of ‘sabotaging peace’ afte...",https://nypost.com/2025/08/28/world-news/uk-eu...
26,34,2025-08-28-20-07-42 +0000,nypost,Transgender Minneapolis shooter Robin Westman ...,https://nypost.com/2025/08/28/us-news/minneapo...
256,33,2025-08-28-00-07-00 +0000,nypost,Kyiv racking up possible bilateral summit loca...,https://nypost.com/2025/08/27/us-news/kyiv-rac...
203,32,2025-08-28-08-02-39 +0000,nypost,"Vladimir Putin, Kim Jong Un to attend Chinese ...",https://nypost.com/2025/08/28/world-news/putin...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
